In [28]:
import pandas as pd
import numpy as np
import datetime

#import xgboost as xgb

from keras.models import Sequential
from keras.layers import Dense
from keras import layers
from keras.layers import Dropout

import matplotlib.pyplot as plt
from tensorflow import keras
import tensorflow as tf

In [16]:
%%capture
from tqdm import tqdm_notebook as tqdm
tqdm().pandas()

In [ ]:
from sklearn.model_selection import train_test_split

labels =  df_train[label_cols].values

train_ids, validation_ids, train_labels, validation_labels = train_test_split(input_ids, labels, random_state=0, test_size=0.1)
train_masks, validation_masks, _, _ = train_test_split(attention_masks, labels, random_state=0, test_size=0.1)

train_size = len(train_inputs)
validation_size = len(validation_inputs)

In [4]:
#this is the raw dataset
data16 = pd.read_csv(r'/Users/cairo/Google Drive/wechat data/016.csv', sep=',', error_bad_lines=False, index_col=False, dtype='unicode')
data16_sample = pd.read_csv(r'/Users/cairo/Google Drive/wechat data/data16_sample.csv', sep=',', error_bad_lines=False, index_col=False, dtype='unicode')

#topicdata = pd.read_csv(r'C:/Users/Junhao/Google Drive/wechat data/TopicOutcomeAll20Topic.csv', sep=',', error_bad_lines=False, index_col=False, dtype='unicode')

In [61]:
#data16_sample = data16.iloc[:1000]
#data16_sample.to_csv("./data16a.csv", sep=',',index=False)

In [6]:
#cutword1 = cutword.head(1000).wordsCut
#cutword3 = cutword.head(1000).title  #i can also try to use only the article titles as bert input 

cutword2 = data16_sample.head(500).content.dropna() #use 1000 article content from rawdata as bert input
#I need to cut each article to the length of 100 characters because the tokenizer can only deal with a max length of 512
#the computation takes too long if the kept article length is long

cutword3 = []
for i in cutword2:
    dd = i[:100]
    cutword3.append(dd)

In [7]:
from transformers import (
   BertTokenizerFast,
   AutoModelForMaskedLM,
   AutoModelForTokenClassification,
)



In [21]:
from transformers import BertTokenizer
from keras.preprocessing.sequence import pad_sequences

bert_model_name = 'bert-base-chinese'

tokenizer = BertTokenizer.from_pretrained(bert_model_name)
MAX_LEN = 128

def tokenize_sentences(sentences, tokenizer, max_seq_len = 128):
    tokenized_sentences = []

    for sentence in tqdm(sentences):
        tokenized_sentence = tokenizer.encode(
                            sentence,                  # Sentence to encode.
                            add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                            max_length = max_seq_len,  # Truncate all sentences.
                    )
        
        tokenized_sentences.append(tokenized_sentence)

    return tokenized_sentences

def create_attention_masks(tokenized_and_padded_sentences):
    attention_masks = []

    for sentence in tokenized_and_padded_sentences:
        att_mask = [int(token_id > 0) for token_id in sentence]
        attention_masks.append(att_mask)

    return np.asarray(attention_masks)

input_ids = tokenize_sentences(cutword3, tokenizer, MAX_LEN)
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", value=0, truncating="post", padding="post")
attention_masks = create_attention_masks(input_ids)

<ipython-input-21-09d75a36d7ef>:12: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for sentence in tqdm(sentences):


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [22]:
input_ids

array([[  101,   791,  1921, ...,     0,     0,     0],
       [  101,   837,  6853, ...,     0,     0,     0],
       [  101,   369, 10839, ...,     0,     0,     0],
       ...,
       [  101,   837,  6853, ...,     0,     0,     0],
       [  101,  3152,   120, ...,     0,     0,     0],
       [  101,  3036,  6206, ...,     0,     0,     0]])

In [23]:
likeCount = data16.head(500).likeCount #use likecount as dependent variable 

df = data16[["likeCount", "content"]].head(500)

removeinds = pd.isnull(df).any(1).to_numpy().nonzero()[0]

likeCount = [i for j, i in enumerate(likeCount) if j not in removeinds] # remove the two likecount where content is empty


In [47]:
from sklearn.model_selection import train_test_split

labels = [float(i) for i in likeCount]

train_ids, validation_ids, train_labels, validation_labels = train_test_split(input_ids, labels, random_state=0, test_size=0.1)
train_masks, validation_masks, _, _ = train_test_split(attention_masks, labels, random_state=0, test_size=0.1)

train_size = len(train_ids)
validation_size = len(validation_ids)

In [48]:
#make the dataset into tensorflow format

def create_dataset(ids, masks, labels):
    def gen():
        for i in range(len(train_ids)):
            yield (
                {
                    "input_ids": ids[i],
                    "attention_mask": masks[i]
                },
                labels[i],
            )

    return tf.data.Dataset.from_generator(
        gen,
        ({"input_ids": tf.int32, "attention_mask": tf.int32}, tf.int64),
        (
            {
                "input_ids": tf.TensorShape([None]),
                "attention_mask": tf.TensorShape([None])
            },
            tf.TensorShape([None]),
        ),
    )

train_dataset = create_dataset(train_ids, train_masks, train_labels)
val_dataset = create_dataset(validation_ids, validation_masks, validation_labels)

In [58]:
from transformers import TFBertForSequenceClassification, BertConfig

model = TFBertForSequenceClassification.from_pretrained(
    bert_model_name, 
    config=BertConfig.from_pretrained(bert_model_name, num_labels=1))

#last_hidden_states = model(train_ids, attention_mask=train_masks)
#y_pred = last_hidden_states[0]
#y_true = validation_labels

# Prepare training: Compile tf.keras model with optimizer, loss and learning rate schedule
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0)

#loss = tf.keras.losses.MSE(y_pred, y_true)
loss = tf.keras.losses.MSE()
metric = tf.keras.metrics.MeanSquaredError()


#try to freeze several layers so they won't be trained
model.layers[0].trainable = False
model.layers[1].trainable = False
model.layers[2].trainable = False
model.layers[3].trainable = False
model.layers[4].trainable = False

model.compile(optimizer=optimizer, loss=loss, metric = metric)

#model.compile(optimizer=optimizer)

# Train and evaluate using tf.keras.Model.fit()
history = model.fit(train_dataset, epochs=1, steps_per_epoch=115, validation_data=val_dataset)

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-chinese and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


TypeError: mean_squared_error() missing 2 required positional arguments: 'y_true' and 'y_pred'